In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ecdc = pd.read_csv('data/ecdc_covid19_20200418.csv', parse_dates=['dateRep'], dayfirst=True)
interest = pd.read_csv('data/interest_over_time.csv', parse_dates=['date'])
trends_title =  pd.read_csv('data/gtrends_dict_trend.csv', parse_dates=['date'])
trends_traffic =  pd.read_csv('data/gtrends_dict_traffic.csv', parse_dates=['date'])
related_queries = pd.read_csv('data/related_queries.csv', parse_dates=['date'])

In [ ]:
ecdc.head()

In [ ]:
interest.head()

In [ ]:
trends_traffic.head()

In [ ]:
trends_title.head()

In [ ]:
related_queries.head()

In [ ]:
# filtering the ecdc dataframe with the restricted countries for our project

countries = interest.Country.unique()

final_df = ecdc.loc[ecdc['countriesAndTerritories'].isin(countries)]

print("final_df shape:",final_df.shape)
final_df.head()

In [ ]:
# cleaning final_df columns

final_df.rename(columns={'dateRep':'date','countriesAndTerritories':'country','geoId':'country_code'},inplace=True)


In [ ]:
# cleaning the interest dataframe to make it matched with final when merging 

interest.rename(columns={'Country':'country'},inplace=True)

In [ ]:
# cleaning the related_queries dataframe for merging

related_queries.drop(columns='Unnamed: 0',inplace=True)
related_queries.rename(columns={'Country':'country'},inplace=True)

In [ ]:
# cleaning of dataframes for merging

trends_traffic.drop(columns='Unnamed: 0',inplace=True)
trends_title.drop(columns='Unnamed: 0',inplace=True)

trends_traffic.rename(columns={'0':'trends_traffic'},inplace=True)
trends_title.rename(columns={'0':'trends_title'},inplace=True)

trends_title.head()

In [ ]:
# Merging interest dataframe

result = pd.merge(final_df, interest, how='left', on=['date','country'])
result.head()

In [ ]:
# Merging related_queries dataframe

result_2 = pd.merge(result, related_queries, how='left', on=['date','country','coronavirus'])
result_2.head()

In [ ]:
# Merging trends_title dataframe

result_3 = pd.merge(result_2, trends_title, how='left', on=['date','country_code'])
result_3.head()

In [ ]:
# Merging trends_traffic dataframe

result_4 = pd.merge(result_3, trends_traffic, how='left', on=['date','country_code'])
result_4.head()

In [ ]:
# saving the final result as csv 

result_4.to_csv('data/final_dataframe.csv')